In [ ]:
a = model.predict(x_test)
np.argmax(a, axis=1)

array([228, 244, 248, ...,  71, 248, 248])

In [ ]:
y_test.shape

NameError: ignored

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, Adadelta, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import load_model
import gen_dataset
import matplotlib.pyplot as plt
import scipy.io as sio

first = 1
accs = []
no_of_traces = []
keys_accuracy = []

#param_range = np.linspace(12000, 46000, 10)
param_range = np.linspace(3500, 3500, 1)
for param in param_range:
    #opts = {"Average": {"n_avg": 1}, "Resize": {"input_size": 1250, "num_traces": int(param)}}
    opts = {"Average": {"n_avg": 1}}
        #"PCA":{"k":250}}
    #D = gen_dataset.gen_dataset(["drive/My Drive/ML_unprotected_chip_AG"], opts) #folder containing the trace file
    D = gen_dataset.gen_dataset(["drive/My Drive/EM_PROBE_ATTACK_DETECTION_NPY"], opts) #folder containing the trace file
    #D = gen_dataset.gen_dataset_new(["drive/My Drive/MLSCA_TVTF_FINAL_SLIDE_NEW_DB"], opts) #folder containing the trace file need to change gen_dataset_new fn 
    #D = gen_dataset.gen_dataset(["ML_ID_1Byte"], opts)  # folder containing the trace file
    #D = gen_dataset.gen_dataset(["MLSCA_DSAC_50K_Traces_2k_avg_with_a_wrong_key"], opts)
    #D = gen_dataset.gen_dataset(["MLSCA_DSAC_NPY"], opts)
    x_train, y_train, x_val, y_val, x_test, y_test, input_size, output_size, model_choice = D
    #model_choice = 'cnn'
    model_dense = tf.keras.models.Sequential([
        tf.keras.layers.Reshape((input_size, ), input_shape=(input_size, )),
        tf.keras.layers.Dense(512, activation=tf.nn.relu,
                              kernel_regularizer=tf.keras.regularizers.l2(0.0)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(.2),
        tf.keras.layers.Dense(300, activation=tf.nn.relu,
                              kernel_regularizer=tf.keras.regularizers.l2(0.0)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(.2),
        tf.keras.layers.Dense(1500, activation=tf.nn.relu,
                              kernel_regularizer=tf.keras.regularizers.l2(0.0)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.01),
        tf.keras.layers.Dense(output_size, activation=tf.nn.softmax)
    ])

    if model_choice == "cnn":
        model_cnn = tf.keras.models.Sequential([
            tf.keras.layers.Reshape((input_size, 1), input_shape=(input_size, )),
            tf.keras.layers.Conv1D(64, 5, activation=tf.nn.relu),
            tf.keras.layers.Conv1D(64, 5, activation=tf.nn.relu),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPool1D((3)),
            tf.keras.layers.Conv1D(128, 5, activation=tf.nn.relu),
            tf.keras.layers.Conv1D(128, 5, activation=tf.nn.relu),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPool1D((3)),
            tf.keras.layers.Conv1D(128, 5, activation=tf.nn.relu),
            tf.keras.layers.Conv1D(128, 5, activation=tf.nn.relu),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPool1D((3)),
            tf.keras.layers.Conv1D(128, 5, activation=tf.nn.relu),
            tf.keras.layers.Conv1D(128, 5, activation=tf.nn.relu),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPool1D((3)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1024, activation=tf.nn.relu,
                                  kernel_regularizer=tf.keras.regularizers.l2(0.0)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(256, activation=tf.nn.relu,
                                  kernel_regularizer=tf.keras.regularizers.l2(0.0)),

            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(.4),
            tf.keras.layers.Dense(256, activation=tf.nn.softmax)
        ])
        model = model_cnn

    if model_choice == "spect_cnn":
        spect_cnn = tf.keras.models.Sequential([
        tf.keras.layers.Reshape(x_train[0].shape + (1,), input_shape=x_train[0].shape),
        tf.keras.layers.Conv2D(250, 3, activation=tf.nn.relu),
        tf.keras.layers.Conv2D(250, 3, activation=tf.nn.relu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D((2, 2)),
        tf.keras.layers.Conv2D(150, 3, activation=tf.nn.relu),
        tf.keras.layers.Conv2D(150, 3, activation=tf.nn.relu),
    #    tf.keras.layers.MaxPool2D((2, 2)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(50, 3, activation=tf.nn.relu),
        tf.keras.layers.Conv2D(50, 3, activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2, 2)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(150, activation=tf.nn.relu,
                              kernel_regularizer=tf.keras.regularizers.l2(0.00)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(256, activation=tf.nn.softmax)
    ])
        model = spect_cnn
    else:
        model = model_dense
    adam = Adam(lr=0.00021)
    adadelta = Adadelta()
    rmsprop = RMSprop(lr=0.00001)
    chkpt = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, min_lr=0.0000001)
    model.compile(optimizer=adam,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()
    #plt.plot(model.get_layer("dense").get_weights()[0])
    #plt.show()

    # if first == 1:
    #     print("saving Model")
    #     model.save("tmp.h5")
    #     first = 0
    # model = load_model("tmp.h5")
    # print("loading model")
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val),
                       epochs=100, shuffle=True, batch_size=64, callbacks=[reduce_lr])

    # model = tf.keras.models.load_model("dev2_only.h5")

    # TEST
    metrics = model.evaluate(x_test, y_test)
    print(metrics)
    accs.append(metrics[1]*100)
    no_of_traces.append(param*4)
    a = model.predict(x_test)
    b = np.argmax(a, axis=1)
    keys_accuracy.append(b)
    #model.save("devN1_4.h5")

    """
    plt.plot(history.history['accuracy'], linewidth=2)
    plt.plot(history.history['val_accuracy'], linewidth=2)
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy (%)', fontweight='bold')
    plt.xlabel('Epoch', fontweight='bold')
    plt.legend(['Train', 'Validation'], loc='lower right')
    plt.show()

    plt.plot(model.get_layer("dense").get_weights()[0])
    plt.show()
    """
    # break

#sio.savemat("test_accuracy_vs_traces_fpga.mat", {"test_accuracy": accs, "no_of_traces": no_of_traces})
sio.savemat("test_accuracy_vs_traces_unprotected_test_part2.mat", {"test_accuracy2": accs, "no_of_traces2": no_of_traces})
sio.savemat("test_accuracy_vs_traces_unprotected_cip_individual_prediction_test_part2.mat", {"keys_accuracy2": keys_accuracy, "no_of_traces2": no_of_traces})
#sio.savemat("test_accuracy_vs_traces_DSAC_100M.mat", {"test_accuracy": accs, "no_of_traces": no_of_traces})
#sio.savemat("test.mat", {"accuracy": accs})
#plt.plot(param_range, accs)
#plt.show()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_4 (Reshape)          (None, 833)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 512)               427008    
_________________________________________________________________
batch_normalization_12 (Batc (None, 512)               2048      
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 300)               153900    
_________________________________________________________________
batch_normalization_13 (Batc (None, 300)               1200      
_________________________________________________________________
dropout_13 (Dropout)         (None, 300)              

In [7]:
y_test

array([101., 101., 101., 103., 103., 103., 103., 101., 101., 101., 103.,
       101., 101., 101., 101., 103., 103., 103., 103., 103., 103., 103.,
       103., 103., 101., 101., 103., 101., 101., 102., 101., 103., 103.,
       103., 101., 103., 101., 103., 101., 103., 101., 101., 101., 101.,
       101., 103., 101., 101., 101., 101., 101., 103., 101., 101., 101.,
       101., 101., 101., 103., 101., 101., 101., 101., 103., 101., 101.,
       103., 103., 101., 101., 101., 101., 103., 103., 103., 103., 103.,
       103., 103., 101., 103., 103., 103., 103., 103., 103., 101., 101.,
       103., 101., 103., 103., 103., 101., 101., 101., 101., 101., 101.,
       101., 101., 101., 101., 101., 101., 101., 103., 101., 101., 101.,
       101., 101., 103., 103., 103., 103., 101., 101., 102., 103., 101.,
       101., 101., 103., 101., 103., 101., 103., 103., 101., 103., 101.,
       103., 101., 103., 101., 101., 101., 101., 103., 103., 103., 101.,
       103., 103., 101., 103., 103., 101., 101., 10

In [1]:
import sys
sys.path.insert(0, 'drive/My Drive/ML_unprotected_chip_AG')
import gen_dataset